In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

# デバイス設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# データ前処理（リサイズ・正規化など）
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNetの平均
                         [0.229, 0.224, 0.225])  # ImageNetの標準偏差
])

# データ読み込み
train_dataset = datasets.ImageFolder('dataset/train', transform=transform)
val_dataset = datasets.ImageFolder('dataset/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# モデル構築（事前学習済みResNet18を使う）
model = models.resnet18(pretrained=True)

# 出力層を2クラスに変更
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# 損失関数と最適化手法
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# 学習ループ
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

# モデルの保存
torch.save(model.state_dict(), "color_judgment_model.pth")


/Users/nozomi0407/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nozomi0407/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 4.6371
Epoch 2/10, Loss: 1.1358
Epoch 3/10, Loss: 0.3184
Epoch 4/10, Loss: 0.1682
Epoch 5/10, Loss: 0.0983
Epoch 6/10, Loss: 0.0414
Epoch 7/10, Loss: 0.1606
Epoch 8/10, Loss: 0.0329
Epoch 9/10, Loss: 0.0236
Epoch 10/10, Loss: 0.0252


In [9]:
from sklearn.metrics import accuracy_score
import numpy as np

# 評価モードに変更
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 精度（Accuracy）のみ出力
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ モデルの精度（Accuracy）: {acc:.2%}")



✅ モデルの精度（Accuracy）: 57.14%


In [7]:
from PIL import Image

def predict_image(image_path, model, transform):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    return predicted.item()  # 0 または 1 を返す

# モデルの読み込み
model.load_state_dict(torch.load("color_judgment_model.pth"))

# 推論
result = predict_image("/Users/nozomi0407/Hack-1_2025/bot/color_judge/class1_36.png", model, transform)
print("彩り：", "なし（class0）" if result == 0 else "良い（class1）")


彩り： 良い（class1）
